In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn import model_selection
import numpy as np

In [ ]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [ ]:
gb = xgb.XGBClassifier(learning_rate =0.0001, n_estimators=300, 
                       max_depth=50, min_child_weight=1,
                       gamma=2,subsample=0.4,
                       colsample_bytree=1,objective= 'multi:softmax',
                       nthread=-1,scale_pos_weight=1,seed=27)

In [ ]:
train=pd.io.parsers.read_csv("train_submissions.csv")
test = pd.io.parsers.read_csv("test_submissions.csv")

In [ ]:
user = pd.io.parsers.read_csv("user_data.csv")
user1 = user.drop(['submission_count','problem_solved','contribution','country','follower_count','last_online_time_seconds','max_rating','rating','rank','registration_time_seconds'], axis=1)
user1['user_id'] = number.fit_transform(user1['user_id'].astype('str'))
x1 = user1.values

user2 = user.drop(['user_id','submission_count','problem_solved','contribution','country','follower_count','last_online_time_seconds','max_rating','rating','registration_time_seconds'], axis=1)
user2['rank'] = number.fit_transform(user2['rank'].astype('str'))
y1 = user2['rank'].ravel()

In [ ]:
problem = pd.io.parsers.read_csv("problem_data.csv")
problem1 = problem.drop(['level_type','points','tags'], axis=1)
problem1['problem_id'] = number.fit_transform(problem1['problem_id'].astype('str'))
x2 = problem1.values

problem2 = problem.drop(['problem_id','points','tags'], axis=1)
problem2['level_type'] = number.fit_transform(problem2['level_type'].astype('str'))
y2 = problem2['level_type'].ravel()

In [ ]:
test1 = test.drop('ID', axis=1)
test1['user_id'] = number.fit_transform(test1['user_id'].astype('str'))
test11 = test1['user_id']
test11 = pd.DataFrame({ 'user_id': test11 })
test1['problem_id'] = number.fit_transform(test1['problem_id'].astype('str'))
test12 = test1['problem_id']
test12 = pd.DataFrame({ 'problem_id': test12 })

In [ ]:
modelu = gb.fit(x1,y1)

In [ ]:
modelp = gb.fit(x2,y2)

In [ ]:
tu = user1.values
tp = problem1.values
teu = test11.values
tep = test12.values

In [ ]:
predictu = modelu.predict(tu)
predictp = modelp.predict(tp)
predicteu = modelu.predict(teu)
predictep = modelu.predict(tep)

In [ ]:
iu = pd.DataFrame({ 'rank': predictu })
ip = pd.DataFrame({ 'level_type': predictp })
train1 = pd.concat([train,iu,ip], axis=1)
train1 = train1.drop('attempts_range', axis=1)
train1['user_id'] = number.fit_transform(train1['user_id'].astype('str'))
train1['problem_id'] = number.fit_transform(train1['problem_id'].astype('str'))
train1 = pd.get_dummies(data=train1, columns=['rank','level_type'])
x=train1.values

In [ ]:
y=train['attempts_range'].ravel()

In [ ]:
ieu = pd.DataFrame({ 'rank': predicteu })
iep = pd.DataFrame({ 'level_type': predictep })
test1 = pd.concat([test,ieu,iep], axis=1)
test1 = test1.drop('ID', axis=1)
test1['user_id'] = number.fit_transform(test1['user_id'].astype('str'))
test1['problem_id'] = number.fit_transform(test1['problem_id'].astype('str'))
test1 = pd.get_dummies(data=test1, columns=['rank','level_type'])
t1=test1.values

In [ ]:
st = model_selection.cross_val_score(gb, x, y, 
                                              cv=3, scoring='accuracy',n_jobs = -1)
st.mean()

In [ ]:
model = gb.fit(x,y)

In [ ]:
predicts = model.predict(t1)

In [ ]:
i1 = pd.DataFrame({ 'attempts_range': predicts })
i3 = test['ID']
i2 = pd.DataFrame({ 'ID': i3 })
enigma = pd.concat([i2, i1], axis=1)
enigma.head()

In [ ]:
enigma.to_csv("enigma.csv", index=False)